In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders.woe import WOEEncoder


In [7]:
train_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")

In [8]:
train_identity

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,NaN,...,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,NaN,...,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,NaN,NaN,0.0,...,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [9]:
train_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")


In [6]:
train_transaction.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv",
#                               index_col = "TransactionID")


In [ ]:
# test_transaction.head()

In [ ]:
# test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv",
#                            index_col = "TransactionID")


In [ ]:
# test_identity.head()

In [ ]:
# test_df = test_transaction.merge(test_identity, how = 'left', left_index = True, right_index = True)

Merge train data into one dataset

In [10]:
train_df = pd.merge(train_transaction, train_identity,on = 'TransactionID', how = 'left')

determine what portion of is fraud is true.

In [11]:
train_df['isFraud'].value_counts(normalize=True)

isFraud
0    0.96501
1    0.03499
Name: proportion, dtype: float64

In [ ]:
# del train_transaction, train_identity, test_transaction, test_identity

In [ ]:
# df

In [ ]:
#y_train = train_df['isFraud'].copy()
#x_train = train_df.drop('isFraud', axis = 1)


In [ ]:
#imp = SimpleImputer(missing_values=np.NaN, strategy="mean")
#print(imp.fit_transform(train_df))

In [ ]:
# x_test = test_df.copy()

In [ ]:
# del test_df, train_df

In [ ]:
# x_train = x_train.fillna(-999)
# x_test = x_test.fillna(-999)

Percentage of data missing

Drop NA data

cross-validation

another approach to numerical and categorical data

In [13]:
cat_features = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
               'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1',
               'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo',
               'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20',
               'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
               'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

exclude = ['TransactionID', 'TransactionDT', 'isFraud']
num_features = [f for f in train_df.columns if (f not in cat_features) & (f not in exclude)]

# drop more than 90% NAs
col_na = train_df.isna().sum()
to_drop = col_na[(col_na / train_df.shape[0]) > 0.9].index

use_cols = [f for f in train_df.columns if f not in to_drop]
cat_features = [f for f in cat_features if f not in to_drop]
num_features = [f for f in num_features if f not in to_drop]

train_df[cat_features] = train_df[cat_features].astype(str)
train_df[num_features] = train_df[num_features].astype(float)
train_df = train_df[use_cols]

fill NA values

In [14]:
median_values = train_df[num_features].median() 
train_df[num_features] = train_df[num_features].fillna(median_values)

# fill categorical NAs with "missing"
train_df[cat_features] = train_df[cat_features].replace("nan", "missing")

train_df.isna().sum().sum()

0

Separating numerical from categorical data.

In [ ]:
#cat_cols_str=['ProductCD', 'card4', 'card6','P_emaildomain', 'R_emaildomain', 
              #'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12',
              #'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
              #'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 
              #'DeviceType', 'DeviceInfo']
#cat_cols_int=[ 'card1','card2','card3','card5','addr1', 'addr2']

preprocessing

In [ ]:
#def process_categorical_columns(temp_ds):
   # for col in cat_cols_str: 
        #temp_ds[col] = temp_ds[col].fillna('NA')
    #    temp_ds[col]= temp_ds[col].replace(0,'NA')
     #   le = preprocessing.LabelEncoder()
      #  le.fit(temp_ds[col])
       # temp_ds[col] = le.transform(temp_ds[col]) 

    #for col in cat_cols_int: 
     #   temp_ds[col] = temp_ds[col].fillna(0) 
      #  temp_ds[col]= temp_ds[col].replace('NA',0)
       # le = preprocessing.LabelEncoder()
        #le.fit(temp_ds[col])
        #temp_ds[col] = le.transform(temp_ds[col]) 
    #return temp_ds

In [ ]:
#train = process_categorical_columns(train_df)
# test = process_categorical_columns(x_test)


In [ ]:
train.head()

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
train = reduce_mem_usage(train)
#test = reduce_mem_usage(test)

splitting data in to train and validation sets

In [15]:
data = train_df.drop(columns=['TransactionID', 'TransactionDT'])

target = 'isFraud'
num_features = data.select_dtypes(include=np.number).columns
cat_features = data.select_dtypes(exclude=np.number).columns

num_features = [f for f in num_features if f != target]

In [17]:
X_train, X_val, Y_train, Y_val = tts(data[num_features+list(cat_features)], 
                                     data['isFraud'], test_size = 0.2)
X_train, X_val, Y_train, Y_val = tts(X_train, Y_train, test_size = 0.2)

print(len(X_train), 'train examples')
print(len(X_val), 'validation examples')
#print(len(X_test), 'test examples')

377945 train examples
94487 validation examples


categorical encoding

In [21]:
to_ohe=[]
to_emb=[]
for c in cat_features:
    if X_train[c].nunique() < 5:
        to_ohe.append(c)
    else:
        to_emb.append(c)

In [22]:
scaler = StandardScaler()

# Categorical with < 5 unique values will be One Hot Encoded
ohe = OneHotEncoder(handle_unknown='ignore')

# Categorical with >= 5 unique values will be encoded using Weight of Evidence
woe = WOEEncoder()

column_trans = ColumnTransformer(
    [ ('scaler',scaler, num_features),
    ('ohe', ohe, to_ohe),
    ('woe', woe, to_emb)], remainder='passthrough', n_jobs=-1)

X_train_transformed = column_trans.fit_transform(X_train, Y_train)
X_val_transformed = column_trans.transform(X_val)
#X_test_transformed = column_trans.transform(test_X)

print(X_train_transformed.shape, X_val_transformed.shape) #,test_X_transformed.shape)

(377945, 459) (94487, 459)


In [23]:
Model = LogisticRegression(penalty='none', C=1).fit(X_train_transformed, Y_train)
y_prob_logit = Model.predict_proba(X_val_transformed)[:, 1]
print(f'ROC-AUC score: {roc_auc_score(Y_val, y_prob_logit):.5f}')

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


ROC-AUC score: 0.88644


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
rf = RandomForestClassifier(n_estimators = 400, max_features = 0.3, min_samples_leaf = 20,
                           n_jobs = -1, verbose = 1)
rf.fit(X_train_transformed, Y_train)
predVal = rf.predict(X_val_transformed)
actVal = Y_val.values
print(metric.accuracy_score(actVal, predVal))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


references for code ideas
Basic Preprocessing by Antons Rubert:https://www.kaggle.com/code/antonsruberts/basic-preprocessing
